In [2]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp

Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 2.70MB/s]


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
df=pd.read_csv('preprocessed_data.csv')

In [5]:
df.head(4)

,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [6]:
def preprocess(x):
  x=x[:-1]
  return x

In [7]:
df['source']=df['source'].apply(preprocess)
df['target']=df['target'].apply(preprocess)

In [8]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [9]:
df.shape

(2000, 2)

In [10]:
def length(text):#for calculating the length of the sentence
    return len(str(text))

In [11]:
df=df[df['source'].apply(length)<170]
df=df[df['target'].apply(length)<200]

In [12]:
df.shape

(1990, 2)

In [12]:
df['target_in'] = '<start> ' + df['target'].astype(str)
df['target_out'] = df['target'].astype(str) + ' <end>'
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
df.head()

,source,target,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?,<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...,<start> Hi! How did your week go? Haven't hear...,Hi! How did your week go? Haven't heard from y...


In [13]:
df=df.drop('target',axis=1)


In [14]:
df.head(4)

,source,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>


In [15]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.01)

In [16]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['target_in']= str(train.iloc[0]['target_in'])+' <end>'
train.iloc[0]['target_out']= str(train.iloc[0]['target_out'])+' <end>'

(1970, 3) (20, 3)


In [17]:
tknizer_source = Tokenizer()
tknizer_source.fit_on_texts(train['source'].values)
tknizer_target = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_target.fit_on_texts(train['target_in'].values)

In [18]:
vocab_size_target=len(tknizer_target.word_index.keys())
print(vocab_size_target)
vocab_size_source=len(tknizer_source.word_index.keys())
print(vocab_size_source)

3039
3708


In [19]:
tknizer_target.word_index['<start>'], tknizer_target.word_index['<end>']

(1, 1447)

In [20]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output=0
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")


    def call(self,input_sequence,states):

      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer oupulstm_state_h,lstm_state_ct to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      
      input_embedd                           = self.embedding(input_sequence)
      lstm_state_h,lstm_state_c= states[0],states[1]
      self.lstm_output,lstm_state_h,lstm_state_c=self.lstm(input_embedd)
      return self.lstm_output,lstm_state_h,lstm_state_c


    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [tf.zeros((batch_size,self.lstm_size)),tf.zeros((batch_size,self.lstm_size))]
      


In [21]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):
    super().__init__()
    self.scoring_function=scoring_function
    # Please go through the reference notebook and research paper to complete the scoring functions

    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      pass
    if self.scoring_function == 'general':
      # Intialize variables needed for General score function here
      self.weight=tf.keras.layers.Dense(att_units)
    elif self.scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
      self.weight1=tf.keras.layers.Dense(att_units)
      self.weight2=tf.keras.layers.Dense(att_units)
      self.v=tf.keras.layers.Dense(1)
  
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        decoder_hidden_state=tf.expand_dims(decoder_hidden_state,axis=2)
        value=tf.matmul(encoder_output,decoder_hidden_state)
    elif self.scoring_function == 'general':
        # Implement General score function here
        decoder_hidden_state=tf.expand_dims(decoder_hidden_state,axis=2)
        value=tf.matmul(self.weight(encoder_output),decoder_hidden_state)
    elif self.scoring_function == 'concat':
        # Implement General score function here
        decoder_hidden_state=tf.expand_dims(decoder_hidden_state,axis=1)
        value=self.v(tf.nn.tanh(self.weight1(decoder_hidden_state)+self.weight2(encoder_output)))
    attention_weights=tf.nn.softmax(value,axis=1)
    context_vector=attention_weights*encoder_output
    return tf.reduce_sum(context_vector,axis=1),attention_weights
    
    
    

In [22]:
class One_Step_Decoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.tar_vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length 
      self.dec_units = dec_units
      self.score_fun = score_fun
      self.att_units = att_units
      # we are using embedding_matrix and not training the embedding layer
      self.embedding = tf.keras.layers.Embedding(input_dim=self.tar_vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", trainable=True)
      self.lstm = tf.keras.layers.LSTM(self.dec_units, return_sequences=True, return_state=True)
      self.dense = tf.keras.layers.Dense(self.tar_vocab_size)
      self.attention = Attention(self.score_fun,self.att_units)

  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    output = self.embedding(input_to_decoder)
    context_vector,attention_weights = self.attention(state_h,encoder_output)
    context_vector1 = tf.expand_dims(context_vector,1)
    concat = tf.concat([output,context_vector1],axis=-1)
    decoder_output,state_h,state_c = self.lstm(concat,initial_state=[state_h,state_c])
    final_output = self.dense(decoder_output)
    final_output = tf.reshape(final_output,(-1,final_output.shape[2]))
    return final_output,state_h,state_c,attention_weights,context_vector




In [23]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super(Decoder,self).__init__()
      self.vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units=dec_units
      self.att_units=att_units
      self.score_fun=score_fun
      self.onestepdecoder=One_Step_Decoder(self.vocab_size,self.embedding_dim,self.input_length,self.dec_units ,self.score_fun ,self.att_units)
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        all_outputs=tf.TensorArray(tf.float32,size=input_to_decoder.shape[1])
        #Create a tensor array as shown in the reference notebook
        #Iterate till the length of the decoder input
        for timestep in range(input_to_decoder.shape[1]):
            # Call onestepdecoder for each token in decoder_input
            output,state_h,state_c,attention_weights,context_vector=self.onestepdecoder(input_to_decoder[:,timestep:timestep+1],encoder_output,decoder_hidden_state,decoder_cell_state)

            # Store the output in tensorarray
            all_outputs=all_outputs.write(timestep,output)
        all_outputs=tf.transpose(all_outputs.stack(),[1,0,2])
        # Return the tensor array
        return all_outputs
        
        
    

In [24]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size,score_fun):
    #Intialize objects from encoder decoder
    super().__init__() # https://stackoverflow.com/a/27134600/4084039
    self.batch_size=batch_size
    self.encoder = Encoder(vocab_size_source+1,300,128,encoder_inputs_length)
    self.decoder = Decoder(vocab_size_target+1,300,decoder_inputs_length,128,score_fun,128)
    
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    input,output = data[0], data[1]
    initial_state=self.encoder.initialize_states(self.batch_size)
    encoder_output, encoder_h, encoder_c = self.encoder(input,initial_state)
    decoder_output= self.decoder(output, encoder_output, encoder_h, encoder_c)
    return decoder_output




In [25]:
#https://www.tensorflow.org/tutorials/text/image_captioning#model
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
def loss_function(real, pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple sparse categorical crossentropy?
    Yes, you can use simple sparse categorical crossentropy as loss like we did in task-1. But in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [26]:
class Dataset:
    def __init__(self, df, tknizer_source, tknizer_target, source_len,target_len):
        self.encoder_inps = df['source'].values
        self.decoder_inps = df['target_in'].values
        self.decoder_outs = df['target_out'].values
        self.tknizer_target = tknizer_target
        self.tknizer_source = tknizer_source
        self.source_len = source_len
        self.target_len = target_len


    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_source.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_target.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_target.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.source_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.target_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [27]:
train_dataset = Dataset(train, tknizer_source, tknizer_target,39,43)
test_dataset  = Dataset(validation, tknizer_source, tknizer_target,39,43)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=20)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 39) (512, 43) (512, 43)


In [28]:
tf.config.experimental_run_functions_eagerly(True)


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [29]:
tf.config.run_functions_eagerly(True)


In [30]:
#Create an object of encoder_decoder Model class, 
# Compile the model and fit the model
# Implement teacher forcing while training your model. You can do it two ways.
# Prepare your data, encoder_input,decoder_input and decoder_output
# if decoder input is 
# <start> Hi how are you
# decoder output should be
# Hi How are you <end>
# i.e when you have send <start>-- decoder predicted Hi, 'Hi' decoder predicted 'How' .. e.t.c

# or
 
# model.fit([train_ita,train_eng],train_eng[:,1:]..)
# Note: If you follow this approach some grader functions might return false and this is fine.
model  = encoder_decoder(encoder_inputs_length=39,decoder_inputs_length=43,output_vocab_size=vocab_size_target,batch_size=512,score_fun="concat")
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer,loss=loss_function)
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//20
model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/100
3/3 [==============================] - 4s 1s/step - loss: 2.7480 - val_loss: 2.1525
Epoch 2/100
3/3 [==============================] - 3s 1s/step - loss: 2.3138 - val_loss: 1.6742
Epoch 3/100
3/3 [==============================] - 3s 1s/step - loss: 2.1388 - val_loss: 1.6743
Epoch 4/100
3/3 [==============================] - 3s 1s/step - loss: 2.1127 - val_loss: 1.6691
Epoch 5/100
3/3 [==============================] - 3s 1s/step - loss: 2.0877 - val_loss: 1.6687
Epoch 6/100
3/3 [==============================] - 3s 997ms/step - loss: 2.0770 - val_loss: 1.6678
Epoch 7/100
3/3 [==============================] - 3s 1s/step - loss: 2.0696 - val_loss: 1.6660
Epoch 8/100
3/3 [==============================] - 3s 1s/step - loss: 2.0689 - val_loss: 1.6673
Epoch 9/100
3/3 [==============================] - 3s 1s/step - loss: 2.0653 - val_loss: 1.6610
Epoch 10/100
3/3 [==============================] - 3s 1s/step - loss: 2.0611 - val_loss: 1.6525
Epoch 11/100
3/3 [==================

KeyboardInterrupt: ignored

In [31]:
batch_size=512
units=128

In [40]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         Save the attention weights
         And get the word using the tokenizer(word index) and then store it in a string.
  E. Call plot_attention(#params)
  F. Return the predicted sentence
  '''
  initial_state_enc=[np.zeros((batch_size,units)),np.zeros((batch_size,units))]
  inp_seq = tknizer_source.texts_to_sequences([input_sentence])
  inp_seq = pad_sequences(inp_seq,padding='post',maxlen=39)

  en_outputs,state_h , state_c = model.layers[0](tf.constant(inp_seq),initial_state_enc)
  cur_vec = tf.constant([[tknizer_target.word_index['<start>']]])
  pred = []
  for i in range(43):
    output,state_h,state_c,attention_weights,context_vector = model.layers[1].onestepdecoder(cur_vec,en_outputs,state_h,state_c)
    cur_vec = np.reshape(np.argmax(output), (1, 1))
    pred.append(tknizer_target.index_word[cur_vec[0][0]])
    if(pred[-1]=='<end>'):
      break
    translated_sentence = ' '.join(pred)
  return translated_sentence

In [41]:
for i in validation['source']:
  print("The Actual Output:")
  print(i)
  predicted=predict(i)
  print("The predicted output is: ")
  print(predicted)
  print('>'*100)

The Actual Output:
Helo k.reen n p.ple.hw r u?care 2 chat any1?
The predicted output is: 
hi care to chat
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output:
Haha... Not accurate right....
The predicted output is: 
ok
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output:
oh cz1102? ya dat was me.i offline liao.im in clubrm ma
The predicted output is: 
hey are you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are you
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output:
eh dont think so... Library can print?
The predicted output is: 
hey i am
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output:
Sigh wat's new man... So t

In [42]:
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
import nltk.translate.bleu_score as bleu
bleu_scores_lst=[]
for i in validation[:]['source']:
  reference = [i.split(),] # the original
  predicted=predict(i)
  translation = predicted.split()
  values=bleu.sentence_bleu(reference, translation)
  bleu_scores_lst.append(values)

# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [43]:
average_bleu_scores=sum(bleu_scores_lst)/len(bleu_scores_lst)
print("Average BLEU score of these 20 test data sentences is: ",average_bleu_scores)

Average BLEU score of these 20 test data sentences is:  0.010138124061366445


In [44]:
bleu_scores_lst


[0.2025894847023147,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.9007028956229227e-06,
 0,
 0,
 0.00017109582211857066,
 0,
 0]

Character_Level:

In [13]:
df=pd.read_csv('preprocessed_data.csv')
df.head()

,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n
4,4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [14]:
def preprocess(x):
  x=x[:-1]
  return x

In [15]:
df['source']=df['source'].apply(preprocess)
df['target']=df['target'].apply(preprocess)

In [16]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [17]:
df.shape

(2000, 2)

In [18]:
def length(text):#for calculating the length of the sentence
    return len(str(text))

In [19]:
df=df[df['source'].apply(length)<170]
df=df[df['target'].apply(length)<200]

In [20]:
df.shape


(1990, 2)

In [21]:
df['target_in'] = '\t' + df['target'].astype(str)
df['target_out'] = df['target'].astype(str) + '\n'
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
df.head()

,source,target,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?,\tDo you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...,\tYeap. You reaching? We ordered some Durian p...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...,\tThey become more expensive already. Mine is ...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?,\tI'm Thai. What do you do?,I'm Thai. What do you do?\n
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...,\tHi! How did your week go? Haven't heard from...,Hi! How did your week go? Haven't heard from y...


In [22]:
df=df.drop('target',axis=1)


In [23]:
df.head(4)

,source,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",\tDo you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,\tYeap. You reaching? We ordered some Durian p...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,\tThey become more expensive already. Mine is ...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,\tI'm Thai. What do you do?,I'm Thai. What do you do?\n


In [24]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.01)

In [25]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['target_in']= str(train.iloc[0]['target_in'])+'\n'
train.iloc[0]['target_out']= str(train.iloc[0]['target_out'])+'\n'

(1970, 3) (20, 3)


In [26]:
tknizer_source = Tokenizer(filters=None,char_level=True,lower=False)
tknizer_source.fit_on_texts(train['source'].values)
tknizer_target = Tokenizer(filters=None,char_level=True,lower=False)
tknizer_target.fit_on_texts(train['target_in'].values)

In [27]:
vocab_size_target=len(tknizer_target.word_index.keys())
print(vocab_size_target)
vocab_size_source=len(tknizer_source.word_index.keys())
print(vocab_size_source)

92
103


In [28]:
tknizer_target.word_index['\t'], tknizer_target.word_index['\n']

(20, 85)

In [29]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output=0
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")


    def call(self,input_sequence,states):

      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer oupulstm_state_h,lstm_state_ct to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      
      input_embedd                           = self.embedding(input_sequence)
      lstm_state_h,lstm_state_c= states[0],states[1]
      self.lstm_output,lstm_state_h,lstm_state_c=self.lstm(input_embedd)
      return self.lstm_output,lstm_state_h,lstm_state_c


    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [tf.zeros((batch_size,self.lstm_size)),tf.zeros((batch_size,self.lstm_size))]
      


In [30]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):
    super().__init__()
    self.scoring_function=scoring_function
    # Please go through the reference notebook and research paper to complete the scoring functions

    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      pass
    if self.scoring_function == 'general':
      # Intialize variables needed for General score function here
      self.weight=tf.keras.layers.Dense(att_units)
    elif self.scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
      self.weight1=tf.keras.layers.Dense(att_units)
      self.weight2=tf.keras.layers.Dense(att_units)
      self.v=tf.keras.layers.Dense(1)
  
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        decoder_hidden_state=tf.expand_dims(decoder_hidden_state,axis=2)
        value=tf.matmul(encoder_output,decoder_hidden_state)
    elif self.scoring_function == 'general':
        # Implement General score function here
        decoder_hidden_state=tf.expand_dims(decoder_hidden_state,axis=2)
        value=tf.matmul(self.weight(encoder_output),decoder_hidden_state)
    elif self.scoring_function == 'concat':
        # Implement General score function here
        decoder_hidden_state=tf.expand_dims(decoder_hidden_state,axis=1)
        value=self.v(tf.nn.tanh(self.weight1(decoder_hidden_state)+self.weight2(encoder_output)))
    attention_weights=tf.nn.softmax(value,axis=1)
    context_vector=attention_weights*encoder_output
    return tf.reduce_sum(context_vector,axis=1),attention_weights
    
    
    

In [31]:
class One_Step_Decoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.tar_vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length 
      self.dec_units = dec_units
      self.score_fun = score_fun
      self.att_units = att_units
      # we are using embedding_matrix and not training the embedding layer
      self.embedding = tf.keras.layers.Embedding(input_dim=self.tar_vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", trainable=True)
      self.lstm = tf.keras.layers.LSTM(self.dec_units, return_sequences=True, return_state=True)
      self.dense = tf.keras.layers.Dense(self.tar_vocab_size)
      self.attention = Attention(self.score_fun,self.att_units)

  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    output = self.embedding(input_to_decoder)
    context_vector,attention_weights = self.attention(state_h,encoder_output)
    context_vector1 = tf.expand_dims(context_vector,1)
    concat = tf.concat([output,context_vector1],axis=-1)
    decoder_output,state_h,state_c = self.lstm(concat,initial_state=[state_h,state_c])
    final_output = self.dense(decoder_output)
    final_output = tf.reshape(final_output,(-1,final_output.shape[2]))
    return final_output,state_h,state_c,attention_weights,context_vector




In [32]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super(Decoder,self).__init__()
      self.vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units=dec_units
      self.att_units=att_units
      self.score_fun=score_fun
      self.onestepdecoder=One_Step_Decoder(self.vocab_size,self.embedding_dim,self.input_length,self.dec_units ,self.score_fun ,self.att_units)
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        all_outputs=tf.TensorArray(tf.float32,size=tf.shape(input_to_decoder)[1])
        #Create a tensor array as shown in the reference notebook
        #Iterate till the length of the decoder input
        for timestep in range(tf.shape(input_to_decoder)[1]):
            # Call onestepdecoder for each token in decoder_input
            output,state_h,state_c,attention_weights,context_vector=self.onestepdecoder(input_to_decoder[:,timestep:timestep+1],encoder_output,decoder_hidden_state,decoder_cell_state)

            # Store the output in tensorarray
            all_outputs=all_outputs.write(timestep,output)
        all_outputs=tf.transpose(all_outputs.stack(),[1,0,2])
        # Return the tensor array
        return all_outputs
        
        
    

In [44]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size,score_fun):
    #Intialize objects from encoder decoder
    super().__init__() # https://stackoverflow.com/a/27134600/4084039
    self.batch_size=batch_size
    self.encoder = Encoder(vocab_size_source+1,300,100,encoder_inputs_length)
    self.decoder = Decoder(vocab_size_target+1,300,decoder_inputs_length,100,score_fun,100)
    
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    input,output = data[0], data[1]
    initial_state=self.encoder.initialize_states(self.batch_size)
    encoder_output, encoder_h, encoder_c = self.encoder(input,initial_state)
    decoder_output= self.decoder(output, encoder_output, encoder_h, encoder_c)
    return decoder_output




In [45]:
#https://www.tensorflow.org/tutorials/text/image_captioning#model
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
def loss_function(real, pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple sparse categorical crossentropy?
    Yes, you can use simple sparse categorical crossentropy as loss like we did in task-1. But in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [46]:
class Dataset:
    def __init__(self, df, tknizer_source, tknizer_target, source_len,target_len):
        self.encoder_inps = df['source'].values
        self.decoder_inps = df['target_in'].values
        self.decoder_outs = df['target_out'].values
        self.tknizer_target = tknizer_target
        self.tknizer_source = tknizer_source
        self.source_len = source_len
        self.target_len = target_len


    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_source.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_target.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_target.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.source_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.target_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [47]:
train_dataset = Dataset(train, tknizer_source, tknizer_target,170,200)
test_dataset  = Dataset(validation, tknizer_source, tknizer_target,170,200)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=20)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 170) (512, 200) (512, 200)


In [48]:
#Create an object of encoder_decoder Model class, 
# Compile the model and fit the model
# Implement teacher forcing while training your model. You can do it two ways.
# Prepare your data, encoder_input,decoder_input and decoder_output
# if decoder input is 
# <start> Hi how are you
# decoder output should be
# Hi How are you <end>
# i.e when you have send <start>-- decoder predicted Hi, 'Hi' decoder predicted 'How' .. e.t.c

# or
 
# model.fit([train_ita,train_eng],train_eng[:,1:]..)
# Note: If you follow this approach some grader functions might return false and this is fine.
model  = encoder_decoder(encoder_inputs_length=170,decoder_inputs_length=200,output_vocab_size=vocab_size_target,batch_size=512,score_fun="concat")
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer,loss=loss_function)
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//20
model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
3/3 [==============================] - 16s 4s/step - loss: 1.4901 - val_loss: 0.9755
Epoch 2/100
3/3 [==============================] - 9s 3s/step - loss: 1.1887 - val_loss: 0.9433
Epoch 3/100
3/3 [==============================] - 9s 3s/step - loss: 1.1406 - val_loss: 0.9031
Epoch 4/100
3/3 [==============================] - 9s 3s/step - loss: 1.0853 - val_loss: 0.8608
Epoch 5/100
3/3 [==============================] - 8s 3s/step - loss: 1.0354 - val_loss: 0.8301
Epoch 6/100
3/3 [==============================] - 9s 3s/step - loss: 0.9966 - val_loss: 0.8033
Epoch 7/100
3/3 [==============================] - 9s 3s/step - loss: 0.9663 - val_loss: 0.7835
Epoch 8/100
3/3 [==============================] - 9s 3s/step - loss: 0.9445 - val_loss: 0.7679
Epoch 9/100
3/3 [==============================] - 9s 3s/step - loss: 0.9295 - val_loss: 0.7558
Epoch 10/100
3/3 [==============================] - 9s 3s/step - loss: 0.9178 - val_loss: 0.7471
Epoch 11/100
3/3 [====================

In [38]:
model.save_weights("model7.h5")
batch_size=512
units=100

In [39]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         Save the attention weights
         And get the word using the tokenizer(word index) and then store it in a string.
  E. Call plot_attention(#params)
  F. Return the predicted sentence
  '''
  initial_state_enc=[np.zeros((batch_size,units)),np.zeros((batch_size,units))]
  inp_seq = tknizer_source.texts_to_sequences([input_sentence])
  inp_seq = pad_sequences(inp_seq,padding='post',maxlen=170)

  en_outputs,state_h , state_c = model.layers[0](tf.constant(inp_seq),initial_state_enc)
  cur_vec = tf.constant([[tknizer_target.word_index['\t']]])
  pred = []
  #Here 20 is the max_length of the sequence
  for i in range(200):
    output,state_h,state_c,attention_weights,context_vector = model.layers[1].onestepdecoder(cur_vec,en_outputs,state_h,state_c)
    cur_vec = np.reshape(np.argmax(output), (1, 1))
    pred.append(tknizer_target.index_word[cur_vec[0][0]])
    if(pred[-1]=='\n'):
      break
    translated_sentence = ''.join(pred)
  return translated_sentence

In [40]:
for i in validation['source']:
  pred=predict(i)
  print(i)
  print(pred)

Hello h r u
Youcht t t t yot thathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathatha
We saw fiona xie at taka... Haha...
Yout t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t
if u are in town. I take taxi myself.
His at athat t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t
U free tmr? Wana watch finding nemo...
Whathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathathath
Ü c da glasses nice n cn sit properly on è nose... It feels comfy then 

In [41]:

# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
import nltk.translate.bleu_score as bleu
bleu_scores_lst=[]
for i in validation[:]['source']:
  reference = [i.split(),] # the original
  predicted=predict(i)
  translation = predicted.split()
  values=bleu.sentence_bleu(reference, translation)
  bleu_scores_lst.append(values)

# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [42]:
average_bleu_scores=sum(bleu_scores_lst)/len(bleu_scores_lst)
print("Average BLEU score of these 20 test data sentences is: ",average_bleu_scores)

Average BLEU score of these 20 test data sentences is:  0.015891448522335924


In [43]:
bleu_scores_lst

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3178289704467185]